# File contents

**1. Test how the buildings are visualized**

**2. Export data for the iTwin Viewer**

# 1. Test how the buildings are visualized

In [ ]:
import pyvista as pv
import numpy as np
from matplotlib.colors import ListedColormap
import json


## Generate the buildings geometry

path = "./data/buildings_3857.geojson"

# Load GeoJSON data from file
with open(path, 'r') as file:
    geojson_data = json.load(file)

# Specify the desired building id
desired_id = 0

# Filter features based on the 'id' property
filtered_features = [feature for feature in geojson_data['features'] if feature['properties']['id'] == desired_id]

# List to store coordinates without the last element and with z-value 0
filtered_coordinates = []
coordinates_with_midpoints = []


# Access the MultiPolygon for each filtered feature
for feature in filtered_features:
    geometry = feature['geometry']
    coordinates = geometry['coordinates']
    height_m = feature['properties']['height_m']
    
    # Assuming MultiPolygon, you can further iterate through polygons if needed
    for polygon in coordinates:
        # Exclude the last element in the coordinate list and add z-value based on height_m
        filtered_polygon = [(x, y, 0) for x, y in polygon[0][:-1]]
        filtered_polygon_height = [(x, y, height_m) for x, y in polygon[0][:-1]]
        
        # Add the filtered coordinates to the list
        filtered_coordinates.extend(filtered_polygon)
        filtered_coordinates.extend(filtered_polygon_height)

    midpoints_ground = []

    for i in range(len(filtered_polygon)):
        midpoint = [(filtered_polygon[i][0] + filtered_polygon[(i + 1) % len(filtered_polygon)][0]) / 2,
                    (filtered_polygon[i][1] + filtered_polygon[(i + 1) % len(filtered_polygon)][1]) / 2,
                    (filtered_polygon[i][2] + filtered_polygon[(i + 1) % len(filtered_polygon)][2]) / 2]
        midpoints_ground.append(midpoint)

    midpoints_height = []

    for i in range(len(filtered_polygon_height)):
        midpoint = [(filtered_polygon_height[i][0] + filtered_polygon_height[(i + 1) % len(filtered_polygon_height)][0]) / 2,
                    (filtered_polygon_height[i][1] + filtered_polygon_height[(i + 1) % len(filtered_polygon_height)][1]) / 2,
                    (filtered_polygon_height[i][2] + filtered_polygon_height[(i + 1) % len(filtered_polygon_height)][2]) / 2]
        midpoints_height.append(midpoint)

    filtered_coordinates.extend(midpoints_ground)
    filtered_coordinates.extend(midpoints_height)

# Convert the filtered_coordinates to a NumPy array
points = np.array(filtered_coordinates)

# Create faces based on a pattern
faces = np.hstack([
    # [4, 0, 1, 2, 3],    # Bottom face (excluded)
    [4, 4, 5, 6, 7],    # Top face
    [4, 0, 4, 5, 1],       # Side face 1
    [4, 1, 5, 6, 2],       # Side face 2
    [4, 2, 6, 7, 3],       # Side face 3
    [4, 3, 7, 4, 0]        # Side face 4
])

# Create the mesh
mesh = pv.PolyData(points, faces)

# Create the regular faces
regular_faces = mesh.regular_faces
print(regular_faces)


## Generate the scalars

path_receivers = "./data/receivers_3857.geojson"

# Load GeoJSON data from file
with open(path_receivers, 'r') as file:
    geojson_data = json.load(file)


# Filter features based on the building ID
building_features = [feature for feature in geojson_data['features'] if feature['properties']['id_bui'] == desired_id]

# Initialize lists to store Lden, Lnight, and coordinates data
scalars_Lden = []
scalars_Lnig = []
# coordinates = []

# Extract information for each feature
for feature in building_features:
    properties = feature['properties']
    # geometry = feature['geometry']['coordinates']
    id_bui = feature['properties']['id_bui']

    # if properties.get('height_m') == 0 or properties.get('height_m') == 15:
    # Extract Lden and Lnight values
    scalars_Lden.append(round(properties['Lden'], 1))
    scalars_Lnig.append(round(properties['Lnight'], 1))
    # coordinates.append(geometry)

# Fill scalars_Lden and scalars_Lnig with up to 8 elements and
# correct scalar sequence for missing receivers on back faces
while len(scalars_Lden) < 8:
    if id_bui in (2, 6, 7, 8, 12, 16, 17, 20, 21, 22, 23, 27, 28, 29, 30, 36, 37, 41, 42, 49, 50, 54, 55, 58, 59, 61, 62, 64, 65, 68, 69, 71):
        # Manual corrections of scalar sequence for missing receivers on back faces of QGIS polygons
        # Adjustments to make the visualization appear more accurate
        # Missing receiver on position 1
        if id_bui in (42, 49, 59, 61, 65, 69, 71):
            if len(scalars_Lden) == 6:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(0, avg_Lden)
                scalars_Lnig.insert(0, avg_Lnig)
            else:
                avg_Lden = round((scalars_Lden[4] + scalars_Lden[-1]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[-1]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(3, avg_Lden)
                scalars_Lnig.insert(3, avg_Lnig)
        # Missing receiver on poistion 2
        if id_bui in (8, 12, 16, 22, 29, 37, 50, 55, 62):
            if len(scalars_Lden) == 6:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(1, avg_Lden)
                scalars_Lnig.insert(1, avg_Lnig)
            else:
                avg_Lden = round((scalars_Lden[4] + scalars_Lden[5]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[4] + scalars_Lnig[5]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(5, avg_Lden)
                scalars_Lnig.insert(5, avg_Lnig)
        # Missing receiver on poistion 3
        if id_bui in (2, 6, 17, 20, 23, 27, 30, 54):
            if len(scalars_Lden) == 6:
                avg_Lden = round((scalars_Lden[1] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[1] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(2, avg_Lden)
                scalars_Lnig.insert(2, avg_Lnig)
            else:
                avg_Lden = round((scalars_Lden[5] + scalars_Lden[6]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[5] + scalars_Lnig[6]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(6, avg_Lden)
                scalars_Lnig.insert(6, avg_Lnig)
        # Missing receiver on position 4
        if id_bui in (7, 21, 28, 36, 41, 58, 64, 68):
            if len(scalars_Lden) == 6:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(3, avg_Lden)
                scalars_Lnig.insert(3, avg_Lnig)
            else:
                avg_Lden = round((scalars_Lden[4] + scalars_Lden[-1]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[4] + scalars_Lnig[-1]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.append(avg_Lden)
                scalars_Lnig.append(avg_Lnig)
    elif id_bui in (9, 10, 14, 15, 18, 25, 32, 35, 38, 40, 44, 45, 52, 67):
        # Missing receiver on position 2 and 4
        if id_bui in (9, 18, 44, 45):
            if len(scalars_Lden) == 4:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(1, avg_Lden)
                scalars_Lnig.insert(1, avg_Lnig)
            if len(scalars_Lden) == 6:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(3, avg_Lden)
                scalars_Lnig.insert(3, avg_Lnig)
            else:
                if len(scalars_Lden) == 5:
                    avg_Lden = round((scalars_Lden[3] + scalars_Lden[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[3] + scalars_Lnig[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(4, avg_Lden)
                    scalars_Lnig.insert(4, avg_Lnig)
                if len(scalars_Lden) == 7:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(6, avg_Lden)
                    scalars_Lnig.insert(6, avg_Lnig)
        # Missing receiver on position 2 and 3
        if id_bui in (10, 38):
            if len(scalars_Lden) == 4:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(1, avg_Lden)
                scalars_Lnig.insert(1, avg_Lnig)
            if len(scalars_Lden) == 6:
                avg_Lden = round((scalars_Lden[1] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[1] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(2, avg_Lden)
                scalars_Lnig.insert(2, avg_Lnig)
            else:
                if len(scalars_Lden) == 5:
                    avg_Lden = round((scalars_Lden[3] + scalars_Lden[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[3] + scalars_Lnig[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(4, avg_Lden)
                    scalars_Lnig.insert(4, avg_Lnig)
                if len(scalars_Lden) == 7:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(6, avg_Lden)
                    scalars_Lnig.insert(6, avg_Lnig)
        # Missing receiver on position 3 and 4
        if id_bui in (14, 35):
            if len(scalars_Lden) == 4:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(2, avg_Lden)
                scalars_Lnig.insert(2, avg_Lnig)
            if len(scalars_Lden) == 6:
                avg_Lden = round((scalars_Lden[1] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[1] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(3, avg_Lden)
                scalars_Lnig.insert(3, avg_Lnig)
            else:
                avg_Lden = round((scalars_Lden[2] + scalars_Lden[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[2] + scalars_Lnig[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.append(avg_Lden)
                scalars_Lnig.append(avg_Lnig)
        # Missing receiver on position 1 and 4
        if id_bui == 15:
            if len(scalars_Lden) == 4:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 20 # 20 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 20 # 20 dB penalty for back faces
                scalars_Lden.insert(0, avg_Lden)
                scalars_Lnig.insert(0, avg_Lnig)
            if len(scalars_Lden) == 6:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[3]) / 2, 1) - 10 # 10 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[3]) / 2, 1) - 10 # 10 dB penalty for back faces
                scalars_Lden.insert(3, avg_Lden)
                scalars_Lnig.insert(3, avg_Lnig)
            else:
                if len(scalars_Lden) == 5:
                    avg_Lden = round((scalars_Lden[3] + scalars_Lden[4]) / 2, 1) - 12 # 12 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[3] + scalars_Lnig[4]) / 2, 1) - 12 # 12 dB penalty for back faces
                    scalars_Lden.insert(3, avg_Lden)
                    scalars_Lnig.insert(3, avg_Lnig)
                if len(scalars_Lden) == 7:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.append(avg_Lden)
                    scalars_Lnig.append(avg_Lnig)
        # Missing receiver on position 1 and 3
        if id_bui in (25, 32, 40, 52, 67):
            if len(scalars_Lden) == 4:
                avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(0, avg_Lden)
                scalars_Lnig.insert(0, avg_Lnig)
            if len(scalars_Lden) == 6:
                avg_Lden = round((scalars_Lden[1] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                avg_Lnig = round((scalars_Lnig[1] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                scalars_Lden.insert(2, avg_Lden)
                scalars_Lnig.insert(2, avg_Lnig)
            else:
                if len(scalars_Lden) == 5:
                    avg_Lden = round((scalars_Lden[3] + scalars_Lden[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[3] + scalars_Lnig[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(3, avg_Lden)
                    scalars_Lnig.insert(3, avg_Lnig)
                if len(scalars_Lden) == 7:
                    avg_Lden = round((scalars_Lden[5] + scalars_Lden[6]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[5] + scalars_Lnig[6]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(6, avg_Lden)
                    scalars_Lnig.insert(6, avg_Lnig)
    else:
        avg_Lden = round(sum(scalars_Lden[-2:]) / 2, 1) - 3 # 3 dB penalty for back faces
        avg_Lnig = round(sum(scalars_Lnig[-2:]) / 2, 1) - 3 # 3 dB penalty for back faces
        scalars_Lden.append(avg_Lden)
        scalars_Lnig.append(avg_Lnig)

def calculate_and_insert_averages(input_list):
    """Calculate the average values for the corner scalars."""
    # Create a new list from the first four elements
    subset_data_ground = input_list[:4]

    # Calculate and store rounded ground averages
    averages_ground = [round((subset_data_ground[i] + subset_data_ground[(i + 3) % 4]) / 2, 1) for i in range(4)]

    # Create a new list from the next four elements
    subset_data_height = input_list[4:8]

    # Calculate and store rounded height averages
    averages_height = [round((subset_data_height[i] + subset_data_height[(i + 3) % 4]) / 2, 1) for i in range(4)]

    # Concatenate the new averages with the original list
    new_values = averages_ground + averages_height + input_list

    return new_values

# Call the function to calculate and insert all averages in front
scalars_Lden = calculate_and_insert_averages(scalars_Lden)
scalars_Lnig = calculate_and_insert_averages(scalars_Lnig)

# Estimate Lden and Leve
def calculate_decibel_levels(Lden, Lnig, day_hours, eve_hours, nig_hours, day_penalty, eve_penalty, nig_penalty):
    Leve = ((Lden * (day_hours + eve_hours + nig_hours) + Lnig * nig_hours) - 
            (day_penalty * day_hours + eve_penalty * eve_hours + nig_penalty * nig_hours)) / (day_hours + eve_hours + nig_hours)

    Lday = (Lden * (day_hours + eve_hours + nig_hours) - 
            (day_penalty * day_hours + eve_penalty * eve_hours)) / (day_hours + eve_hours)

    return round(Leve, 1), round(Lday, 1)

# Additional parameters
# Hours taken from https://www.umgebungslaerm.nrw.de/laermkartierung/format-und-inhalt
day_hours = 12
eve_hours = 4
nig_hours = 8
# Penalties taken from opeNoise and https://www.umgebungslaerm.nrw.de/laermkartierung/format-und-inhalt
day_penalty = 0
eve_penalty = 5 # dB
nig_penalty = 10 # dB

# Calculate Leve and Lday for all points
scalars_Leve = []
scalars_Lday = []
for Lden, Lnig in zip(scalars_Lden, scalars_Lnig):
    Leve, Lday = calculate_decibel_levels(Lden, Lnig, day_hours, eve_hours, nig_hours, day_penalty, eve_penalty, nig_penalty)
    scalars_Leve.append(Leve)
    scalars_Lday.append(Lday)

# Create color maps
def create_mesh_and_colormap(mesh, scalar_name, scalars):
    # Add the scalar list to the point data
    mesh.point_data[scalar_name] = scalars

    # Define the colors according to DIN 18005-2:1991 (Germany)
    # and apply normalization (0-1)
    dark_blue = np.array([19/256, 67/256, 103/256])     # >80    [19, 67, 103]
    blue = np.array([24/256, 85/256, 140/256])          # >75-80 [24, 85, 140]
    purple = np.array([136/256, 73/256, 123/256])       # >70-75 [136, 73, 123]
    dark_red = np.array([141/256, 26/256, 39/256])      # >65-70 [141, 26, 39]
    red = np.array([199/256, 25/256, 50/256])           # >60-65 [199, 25, 50]
    orange = np.array([239/256, 121/256, 38/256])       # >55-60 [239, 121 , 38]
    brown = np.array([159/256, 111/256, 44/256])        # >50-55 [159, 111, 44]
    yellow = np.array([236/256, 215/256, 33/256])       # >45-50 [236, 215, 33]
    dark_green = np.array([14/256, 76/256, 60/256])     # >40-45 [14, 76, 60]
    green = np.array([29/256, 132/256, 53/256])         # >35-40 [29, 132, 53]
    light_green = np.array([183/256, 206/256, 142/256]) # <=35   [183, 206, 142]

    mapping = np.linspace(mesh[scalar_name].min(), mesh[scalar_name].max(), 256)
    din_colors = np.empty((256, 3))
    din_colors[mapping >  80] = dark_blue
    din_colors[mapping <= 80] = blue
    din_colors[mapping <= 75] = purple
    din_colors[mapping <= 70] = dark_red
    din_colors[mapping <= 65] = red
    din_colors[mapping <= 60] = orange
    din_colors[mapping <= 55] = brown
    din_colors[mapping <= 50] = yellow
    din_colors[mapping <= 45] = dark_green
    din_colors[mapping <= 40] = green
    din_colors[mapping <= 35] = light_green

    # Make the colormap from the listed colors
    colormap = ListedColormap(din_colors)

    return mesh.copy(), colormap

# Example usage
mesh_Lden, colormap_Lden = create_mesh_and_colormap(mesh, 'Lden', scalars_Lden)
mesh_Lday, colormap_Lday = create_mesh_and_colormap(mesh, 'Lday', scalars_Lday)
mesh_Leve, colormap_Leve = create_mesh_and_colormap(mesh, 'Leve', scalars_Leve)
mesh_Lnig, colormap_Lnig = create_mesh_and_colormap(mesh, 'Lnight', scalars_Lnig)

## 1.1 Visualize the output

In [ ]:
pl = pv.Plotter(shape=(1, 2))

color_names = ['red', 'blue', 'green', 'purple', 'paraview', 'orange', 'pink', 'brown', 'darkred', 'darkblue', 'darkgreen', 'rebeccapurple', 'antiquewhite', 'darkorange', 'deeppink', 'sandybrown']

pl.subplot(0, 0)
pl.add_mesh(mesh, color=True, show_edges=True)
for i, point in enumerate(points):
    pl.add_points(points[i], color=color_names[i], point_size=15)
    pl.add_points(points[i], color=color_names[i], point_size=15)

pl.subplot(0, 1)
for i, point in enumerate(points):
    label= f"{str(points[i])}        {(scalars_Lden[i])} dB"
    pl.add_points(points[i], color=color_names[i], label=label)

pl.add_legend(size=(1, 1), loc="center")


pl.link_views()
pl.show()

In [ ]:
# Plot the buildings and a custom cmap for coloring
pl = pv.Plotter(shape=(2, 2))

pl.subplot(0, 0)
pl.add_mesh(mesh, scalars='Lden', show_edges=True, show_scalar_bar=True, 
            cmap=colormap_Lden, lighting=True,
            ambient=0.3,)

pl.subplot(0, 1)
pl.add_mesh(mesh_Lday, scalars='Lday', show_edges=True, show_scalar_bar=True, 
           cmap=colormap_Lday, lighting=True, 
           ambient=0.3,)

pl.subplot(1, 0)
pl.add_mesh(mesh_Leve, scalars='Leve', show_edges=True, show_scalar_bar=True, 
           cmap=colormap_Leve, lighting=True, 
           ambient=0.3,)

pl.subplot(1, 1)
pl.add_mesh(mesh_Lnig, scalars='Lnight', show_edges=True, show_scalar_bar=True, 
           cmap=colormap_Lnig, lighting=True, 
           ambient=0.3,)

pl.link_views()
pl.show()

# 2. Export data for the iTwin Viewer

In [ ]:
import json
import numpy as np
import pyvista as pv
from itertools import islice

def generate_smaller_buildings(geojson_file_path, output_file_path):
    with open(geojson_file_path, 'r') as file:
        geojson_data_smaller = file.read()

    geojson_obj = json.loads(geojson_data_smaller)

    # Iterate through features and modify coordinates
    for feature in geojson_obj['features']:
        coordinates = feature['geometry']['coordinates'][0][0]

        # Modify coordinates
        for i in range(len(coordinates)):
            coord = coordinates[i]
            if i == 0:
                coordinates[i] = [coord[0] - 0.02, coord[1] + 0.02]
            elif i == 1:
                coordinates[i] = [coord[0] + 0.02, coord[1] + 0.02]
            elif i == 2:
                coordinates[i] = [coord[0] + 0.02, coord[1] - 0.02]
            elif i == 3:
                coordinates[i] = [coord[0] - 0.02, coord[1] - 0.02]

    json_output_buildings_smaller = json.dumps(geojson_obj, indent=2)
    with open(output_file_path, 'w') as file:
        file.write(json_output_buildings_smaller)

    print(f"Modified GEOJSON data for smaller buildings has been written to '{output_file_path}'.")

def load_geojson(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def filter_building_features(geojson_data, desired_id):
    return [feature for feature in geojson_data['features'] if feature['properties']['id'] == desired_id]

def extract_building_data(filtered_features):
    filtered_coordinates = []

    for feature in filtered_features:
        geometry = feature['geometry']
        coordinates = geometry['coordinates']
        height_m = feature['properties']['height_m']

        for polygon in coordinates:
            filtered_polygon = [(round(x, 2), round(y, 2), 0) for x, y in polygon[0][:-1]]
            filtered_polygon_height = [(round(x, 2), round(y, 2), height_m) for x, y in polygon[0][:-1]]
            
            filtered_coordinates.extend(filtered_polygon)
            filtered_coordinates.extend(filtered_polygon_height)

        midpoints_ground = []

        for i in range(len(filtered_polygon)):
            midpoint = [round((filtered_polygon[i][0] + filtered_polygon[(i + 1) % len(filtered_polygon)][0]) / 2, 2),
                        round((filtered_polygon[i][1] + filtered_polygon[(i + 1) % len(filtered_polygon)][1]) / 2, 2),
                        round((filtered_polygon[i][2] + filtered_polygon[(i + 1) % len(filtered_polygon)][2]) / 2, 2)]
            midpoints_ground.append(midpoint)
        
        midpoints_height = []

        for i in range(len(filtered_polygon_height)):
            midpoint = [round((filtered_polygon_height[i][0] + filtered_polygon_height[(i + 1) % len(filtered_polygon_height)][0]) / 2, 2),
                        round((filtered_polygon_height[i][1] + filtered_polygon_height[(i + 1) % len(filtered_polygon_height)][1]) / 2, 2),
                        round((filtered_polygon_height[i][2] + filtered_polygon_height[(i + 1) % len(filtered_polygon_height)][2]) / 2, 2)]
            midpoints_height.append(midpoint)

        filtered_coordinates.extend(midpoints_ground)
        filtered_coordinates.extend(midpoints_height)

    return np.array(filtered_coordinates)

def create_mesh(points):
    faces = np.hstack([
        [4, 4, 5, 6, 7],
        [4, 0, 4, 5, 1],
        [4, 1, 5, 6, 2],
        [4, 2, 6, 7, 3],
        [4, 3, 7, 4, 0]
    ])

    mesh = pv.PolyData(points, faces)
    return mesh.regular_faces

def filter_receiver_features(geojson_data, desired_building_id):
    return [feature for feature in geojson_data['features'] if feature['properties']['id_bui'] == desired_building_id]

def calculate_and_insert_averages(input_list):
    subset_data_ground = input_list[:4]
    averages_ground = [round((subset_data_ground[i] + subset_data_ground[(i + 3) % 4]) / 2, 1) for i in range(4)]
    subset_data_height = input_list[4:8]
    averages_height = [round((subset_data_height[i] + subset_data_height[(i + 3) % 4]) / 2, 1) for i in range(4)]
    
    new_values = averages_ground + averages_height + input_list

    return new_values

def extract_scalar_values(building_features):
    scalars_Lden = [round(feature['properties']['Lden'], 1) for feature in building_features]
    scalars_Lnig = [round(feature['properties']['Lnight'], 1) for feature in building_features]
    id_bui = building_features[0]['properties']['id_bui']

    # Fill scalars_Lden and scalars_Lnig with up to 8 elements and
    # correct scalar sequence for missing receivers on back faces
    while len(scalars_Lden) < 8:
        if id_bui in (2, 6, 7, 8, 12, 16, 17, 20, 21, 22, 23, 27, 28, 29, 30, 36, 37, 41, 42, 49, 50, 54, 55, 58, 59, 61, 62, 64, 65, 68, 69, 71):
            # Missing receiver on position 1
            if id_bui in (42, 49, 59, 61, 65, 69, 71):
                if len(scalars_Lden) == 6:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(0, avg_Lden)
                    scalars_Lnig.insert(0, avg_Lnig)
                else:
                    avg_Lden = round((scalars_Lden[4] + scalars_Lden[-1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[-1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(3, avg_Lden)
                    scalars_Lnig.insert(3, avg_Lnig)
            # Missing receiver on poistion 2
            if id_bui in (8, 12, 16, 22, 29, 37, 50, 55, 62):
                if len(scalars_Lden) == 6:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(1, avg_Lden)
                    scalars_Lnig.insert(1, avg_Lnig)
                else:
                    avg_Lden = round((scalars_Lden[4] + scalars_Lden[5]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[4] + scalars_Lnig[5]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(5, avg_Lden)
                    scalars_Lnig.insert(5, avg_Lnig)
            # Missing receiver on poistion 3
            if id_bui in (2, 6, 17, 20, 23, 27, 30, 54):
                if len(scalars_Lden) == 6:
                    avg_Lden = round((scalars_Lden[1] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[1] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(2, avg_Lden)
                    scalars_Lnig.insert(2, avg_Lnig)
                else:
                    avg_Lden = round((scalars_Lden[5] + scalars_Lden[6]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[5] + scalars_Lnig[6]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(6, avg_Lden)
                    scalars_Lnig.insert(6, avg_Lnig)
            # Missing receiver on position 4
            if id_bui in (7, 21, 28, 36, 41, 58, 64, 68):
                if len(scalars_Lden) == 6:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(3, avg_Lden)
                    scalars_Lnig.insert(3, avg_Lnig)
                else:
                    avg_Lden = round((scalars_Lden[4] + scalars_Lden[-1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[4] + scalars_Lnig[-1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.append(avg_Lden)
                    scalars_Lnig.append(avg_Lnig)
        elif id_bui in (9, 10, 14, 15, 18, 25, 32, 35, 38, 40, 44, 45, 52, 67):
            # Missing receiver on position 2 and 4
            if id_bui in (9, 18, 44, 45):
                if len(scalars_Lden) == 4:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(1, avg_Lden)
                    scalars_Lnig.insert(1, avg_Lnig)
                if len(scalars_Lden) == 6:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(3, avg_Lden)
                    scalars_Lnig.insert(3, avg_Lnig)
                else:
                    if len(scalars_Lden) == 5:
                        avg_Lden = round((scalars_Lden[3] + scalars_Lden[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                        avg_Lnig = round((scalars_Lnig[3] + scalars_Lnig[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                        scalars_Lden.insert(4, avg_Lden)
                        scalars_Lnig.insert(4, avg_Lnig)
                    if len(scalars_Lden) == 7:
                        avg_Lden = round((scalars_Lden[0] + scalars_Lden[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                        avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                        scalars_Lden.insert(6, avg_Lden)
                        scalars_Lnig.insert(6, avg_Lnig)
            # Missing receiver on position 2 and 3
            if id_bui in (10, 38):
                if len(scalars_Lden) == 4:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(1, avg_Lden)
                    scalars_Lnig.insert(1, avg_Lnig)
                if len(scalars_Lden) == 6:
                    avg_Lden = round((scalars_Lden[1] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[1] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(2, avg_Lden)
                    scalars_Lnig.insert(2, avg_Lnig)
                else:
                    if len(scalars_Lden) == 5:
                        avg_Lden = round((scalars_Lden[3] + scalars_Lden[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                        avg_Lnig = round((scalars_Lnig[3] + scalars_Lnig[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                        scalars_Lden.insert(4, avg_Lden)
                        scalars_Lnig.insert(4, avg_Lnig)
                    if len(scalars_Lden) == 7:
                        avg_Lden = round((scalars_Lden[0] + scalars_Lden[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                        avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                        scalars_Lden.insert(6, avg_Lden)
                        scalars_Lnig.insert(6, avg_Lnig)
            # Missing receiver on position 3 and 4
            if id_bui in (14, 35):
                if len(scalars_Lden) == 4:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(2, avg_Lden)
                    scalars_Lnig.insert(2, avg_Lnig)
                if len(scalars_Lden) == 6:
                    avg_Lden = round((scalars_Lden[1] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[1] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(3, avg_Lden)
                    scalars_Lnig.insert(3, avg_Lnig)
                else:
                    avg_Lden = round((scalars_Lden[2] + scalars_Lden[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[2] + scalars_Lnig[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.append(avg_Lden)
                    scalars_Lnig.append(avg_Lnig)
            # Missing receiver on position 1 and 4
            if id_bui == 15:
                if len(scalars_Lden) == 4:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 20 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 20 # 3 dB penalty for back faces
                    scalars_Lden.insert(0, avg_Lden)
                    scalars_Lnig.insert(0, avg_Lnig)
                if len(scalars_Lden) == 6:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[3]) / 2, 1) - 10 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[3]) / 2, 1) - 10 # 3 dB penalty for back faces
                    scalars_Lden.insert(3, avg_Lden)
                    scalars_Lnig.insert(3, avg_Lnig)
                else:
                    if len(scalars_Lden) == 5:
                        avg_Lden = round((scalars_Lden[3] + scalars_Lden[4]) / 2, 1) - 12 # 3 dB penalty for back faces
                        avg_Lnig = round((scalars_Lnig[3] + scalars_Lnig[4]) / 2, 1) - 12 # 3 dB penalty for back faces
                        scalars_Lden.insert(3, avg_Lden)
                        scalars_Lnig.insert(3, avg_Lnig)
                    if len(scalars_Lden) == 7:
                        avg_Lden = round((scalars_Lden[0] + scalars_Lden[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                        avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[3]) / 2, 1) - 3 # 3 dB penalty for back faces
                        scalars_Lden.append(avg_Lden)
                        scalars_Lnig.append(avg_Lnig)
            # Missing receiver on position 1 and 3
            if id_bui in (25, 32, 40, 52, 67):
                if len(scalars_Lden) == 4:
                    avg_Lden = round((scalars_Lden[0] + scalars_Lden[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[0] + scalars_Lnig[1]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(0, avg_Lden)
                    scalars_Lnig.insert(0, avg_Lnig)
                if len(scalars_Lden) == 6:
                    avg_Lden = round((scalars_Lden[1] + scalars_Lden[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    avg_Lnig = round((scalars_Lnig[1] + scalars_Lnig[2]) / 2, 1) - 3 # 3 dB penalty for back faces
                    scalars_Lden.insert(2, avg_Lden)
                    scalars_Lnig.insert(2, avg_Lnig)
                else:
                    if len(scalars_Lden) == 5:
                        avg_Lden = round((scalars_Lden[3] + scalars_Lden[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                        avg_Lnig = round((scalars_Lnig[3] + scalars_Lnig[4]) / 2, 1) - 3 # 3 dB penalty for back faces
                        scalars_Lden.insert(3, avg_Lden)
                        scalars_Lnig.insert(3, avg_Lnig)
                    if len(scalars_Lden) == 7:
                        avg_Lden = round((scalars_Lden[5] + scalars_Lden[6]) / 2, 1) - 3 # 3 dB penalty for back faces
                        avg_Lnig = round((scalars_Lnig[5] + scalars_Lnig[6]) / 2, 1) - 3 # 3 dB penalty for back faces
                        scalars_Lden.insert(6, avg_Lden)
                        scalars_Lnig.insert(6, avg_Lnig)           
        else:
            avg_Lden = round(sum(scalars_Lden[-2:]) / 2, 1) - 3
            avg_Lnig = round(sum(scalars_Lnig[-2:]) / 2, 1) - 3
            scalars_Lden.append(avg_Lden)
            scalars_Lnig.append(avg_Lnig)
    
    scalars_Lden = calculate_and_insert_averages(scalars_Lden)
    scalars_Lnig = calculate_and_insert_averages(scalars_Lnig)
    
    return scalars_Lden, scalars_Lnig

def calculate_decibel_levels(Lden, Lnig, day_hours, eve_hours, nig_hours, day_penalty, eve_penalty, nig_penalty):
    Leve = ((Lden * (day_hours + eve_hours + nig_hours) + Lnig * nig_hours) - 
            (day_penalty * day_hours + eve_penalty * eve_hours + nig_penalty * nig_hours)) / (day_hours + eve_hours + nig_hours)

    Lday = (Lden * (day_hours + eve_hours + nig_hours) - 
            (day_penalty * day_hours + eve_penalty * eve_hours)) / (day_hours + eve_hours)

    return round(Leve, 1), round(Lday, 1)

def create_building_channels(Lden, Lday, Leve, Lnig):
    return [
        {"dataType": 0, "inputName": "", "name": "Animation", "data": [{"input": 0.0, "values": Lden}, {"input": 1.0, "values": Lday}, {"input": 2.0, "values": Leve}, {"input": 3.0, "values": Lnig}]},
        {"dataType": 0, "inputName": "", "name": "Lden", "data": [{"input": 0.0, "values": Lden}]},
        {"dataType": 0, "inputName": "", "name": "Lday", "data": [{"input": 0.0, "values": Lday}]},
        {"dataType": 0, "inputName": "", "name": "Levening", "data": [{"input": 0.0, "values": Leve}]},
        {"dataType": 0, "inputName": "", "name": "Lnight", "data": [{"input": 0.0, "values": Lnig}]},
    ]

def process_geojson_buildings(buildings_file_path, receivers_file_path, json_path, day_hours=12, eve_hours=4, nig_hours=8, day_penalty=0, eve_penalty=5, nig_penalty=10):
    building_geojson = load_geojson(buildings_file_path)
    receiver_geojson = load_geojson(receivers_file_path)

    all_buildings_data = {}

    for building_feature in building_geojson['features']:
        building_id = building_feature['properties']['id']

        # Filter building features
        building_features = filter_building_features(building_geojson, building_id)

        # Extract building data
        building_points = extract_building_data(building_features)

        # Create mesh and get regular faces
        regular_faces = create_mesh(building_points)

        # Increase regular_faces integers by +1
        increased_array = [[num + 1 for num in row] for row in regular_faces.tolist()]

        # Flatten the array and join elements with the delimiter
        point_index = [item for sublist in increased_array for item in (sublist + [0])]

        # Filter receiver features
        receiver_features = filter_receiver_features(receiver_geojson, building_id)

        # Extract scalar values
        scalars_Lden, scalars_Lnig = extract_scalar_values(receiver_features)

        # Calculate Leve and Lday for all points
        scalars_Leve = []
        scalars_Lday = []
        for Lden, Lnig in zip(scalars_Lden, scalars_Lnig):
            Leve, Lday = calculate_decibel_levels(Lden, Lnig, day_hours, eve_hours, nig_hours, day_penalty, eve_penalty, nig_penalty)
            scalars_Leve.append(Leve)
            scalars_Lday.append(Lday)

        # Create channels for the building
        building_channels = create_building_channels(scalars_Lden, scalars_Lday, scalars_Leve, scalars_Lnig)

        # Create building data structure
        building_data_structure = {
            f"building_{building_id}": {
                "indexedMesh": {
                    "auxData": {"channels": building_channels, "indices": point_index},
                    "point": building_points.tolist(),
                    "pointIndex": point_index,
                }
            }
        }

        # Update all_buildings_data dictionary
        all_buildings_data.update(building_data_structure)

    # Write the result to a JSON file
    json_output = json.dumps(all_buildings_data, indent=2)
    with open(json_path, 'w') as file:
        file.write(json_output)

    print(f"Processed GEOJSON data to JSON for each building has been written to '{json_path}'.")

def process_json_buildings(input_json_path, output_json_path, index_name, ranges):
    with open(input_json_path, "r") as file:
        data = json.load(file)

    # Create a dictionary to store the JSON data for each mesh
    mesh_data = {}

    # Iterate through each range
    for mesh_index, (start_index, end_index) in enumerate(ranges, start=1):
        sliced_data = {key: value for key, value in islice(data.items(), start_index, end_index)}

        all_lden_values = []
        all_lday_values = []
        all_leve_values = []
        all_lnig_values = []
        all_point_values = []

        # Process data for each building in the range
        for building_data in sliced_data.values():
            if index_name == "buildings_small":
                for points in building_data["indexedMesh"]["point"]:
                    if points[2] < 237: # Add 236.99 meters to z
                        points[2] += 236.99
                    all_point_values.append(points)
            else:
                for channel in building_data["indexedMesh"]["auxData"]["channels"]:
                    if channel["name"] == "Lden":
                        lden_values = channel["data"][0]["values"]
                        all_lden_values.extend(lden_values)
                    if channel["name"] == "Lday":
                        lday_values = channel["data"][0]["values"]
                        all_lday_values.extend(lday_values)
                    if channel["name"] == "Levening":
                        leve_values = channel["data"][0]["values"]
                        all_leve_values.extend(leve_values)
                    if channel["name"] == "Lnight":
                        lnig_values = channel["data"][0]["values"]
                        all_lnig_values.extend(lnig_values)
                for points in building_data["indexedMesh"]["point"]:
                    if points[2] < 237: # Add 237 meters to z
                        points[2] += 237.0
                    all_point_values.append(points)

        def generate_index_lists(num_iterations, original_index):
            # Initialize the first list
            index_lists = [original_index]
            # Generate subsequent lists based on the rule
            for _ in range(num_iterations):
                # Iterate through the previous list and
                # increase the value by 16 
                # (depending on the number of the 
                # original building generation points)
                # then append to the new list
                # If the value is a delimiter, append 0
                new_index = [val + 16 if val != 0 else 0 for val in index_lists[-1]]
                index_lists.append(new_index)

            # Flatten the nested list structure
            return [item for sublist in index_lists for item in sublist]

        num_iterations = len(sliced_data) - 1 # -1 beause starting value original index list (e.g. original_index)
        original_index = data["building_0"]["indexedMesh"]["pointIndex"]
        all_point_index_values = generate_index_lists(num_iterations, original_index)

        if index_name == "buildings_small":
            all_indices_values = []
        else:
            all_indices_values = all_point_index_values

        # Create building data structure
        building_data_one_polyface = {
        "indexedMesh": {
            "auxData": {
                "channels": [
                {
                    "dataType": 0,
                    "inputName": "",
                    "name": "Animation",
                    "data": [ {
                            "input": 0.0,
                            "values": all_lday_values
                        }, {
                            "input": 1.0,
                            "values": all_leve_values
                        }, {
                            "input": 2.0,
                            "values": all_lnig_values
                        }
                    ]
                }, 
                {
                    "dataType": 0,
                    "inputName": "",
                    "name": "Lden",
                    "data": [{
                            "input": 0.0,
                            "values": all_lden_values
                        }
                    ]
                }, 
                {
                    "dataType": 0,
                    "inputName": "",
                    "name": "Lday",
                    "data": [{
                            "input": 0.0,
                            "values": all_lday_values
                        }
                    ]
                }, 
                {
                    "dataType": 0,
                    "inputName": "",
                    "name": "Levening",
                    "data": [{
                            "input": 0.0,
                            "values": all_leve_values
                        }
                    ]
                }, 
                {
                    "dataType": 0,
                    "inputName": "",
                    "name": "Lnight",
                    "data": [{
                            "input": 0.0,
                            "values": all_lnig_values
                        }
                    ]
                }
            ], 
                "indices": all_indices_values},
            "point": all_point_values,
            "pointIndex": all_point_index_values,
        }
        }

        if index_name == "buildings" or index_name == "buildings_small":
            mesh_data = building_data_one_polyface
        else:
            mesh_data[f"{index_name}_{mesh_index:02d}"] = building_data_one_polyface

    json_output = json.dumps(mesh_data, indent=2)
    with open(output_json_path, 'w') as file:
        file.write(json_output)

    print(f"Restructured JSON for the iTwin has been written to '{output_json_path}'.")

def generate_ts_files(input_json_path, output_ts_path, ts_string):
    with open(input_json_path, "r") as file:
        json_data = json.load(file) 
    
    ts_output = json.dumps(json_data, indent=2)
    with open(output_ts_path, 'w') as file:
        file.write(f"{ts_string} = {ts_output}")
    
    print(f"JSON for the iTwin to TS has been written to '{output_ts_path}'.")

# Define file paths
buildings_file_path = "data/buildings_3857.geojson"
receivers_file_path = "data/receivers_3857.geojson"
all_buildings_json_path = 'data/all_buildings.json'
buildings_smaller_file_path = "data/buildings_3857_smaller.geojson"
receivers_smaller_file_path = "data/receivers_3857.geojson"
all_buildings_smaller_json_path = 'data/all_buildings_smaller.json'

# Define the file paths and ranges for all buildings, buildings smaller, buildings clusters, and buildings groups
buildings_input = "data/all_buildings.json"
buildings_output = "data/all_buildings_one_polyface.json"
buildings_index_name = "buildings"
buildings_ranges = [(0, 72)]
buildings_smaller_input = "data/all_buildings_smaller.json"
buildings_smaller_output = "data/all_buildings_one_polyface_smaller.json"
buildings_smaller_index_name = "buildings_small"
buildings_smaller_ranges = [(0, 72)]
buildings_clusters_input = "data/all_buildings.json"
buildings_clusters_output = "data/all_buildings_clusters.json"
buildings_clusters_index_name = "cluster"
cluster_ranges = [
    (0, 11),
    (11, 26),
    (26, 41),
    (41, 48),
    (48, 54),
    (54, 60),
    (60, 66),
    (66, 72)
]
buildings_grouped_input = "data/all_buildings.json"
buildings_grouped_output = "data/all_buildings_groups.json"
buildings_grouped_index_name = "buildings_groups"
buildings_grouped_ranges = [
    # Cluster 1
    (0, 1),      # Building 1
    (1, 2),      # Building 2
    (2, 4),      # Building 3
    (4, 5),      # Building 4
    (5, 6),      # Building 5
    (6, 11),     # Building 6
    # Cluster 2
    (11, 13),     # Building 7
    (13, 14),     # Building 8
    (14, 19),     # Building 9
    (19, 20),     # Building 10
    (20, 26),     # Building 11
    # Cluster 3
    (26, 27),     # Building 12
    (27, 33),     # Building 13
    (33, 34),     # Building 14
    (34, 35),     # Building 15
    (35, 41),     # Building 16
    # Cluster 4
    (41, 46),     # Building 17
    (46, 47),     # Building 18
    (47, 48),     # Building 19
    # Cluster 5
    (48, 51),     # Building 20
    (51, 54),     # Building 21
    (54, 57),     # Building 22
    (57, 60),     # Building 23
    # Cluster 6
    (60, 63),     # Building 24
    (63, 66),     # Building 25
    (66, 70),     # Building 26
    (70, 72),     # Building 27
]

# Define the file paths for .ts files
buildings_ts_output = "../../src/components/scientific-viz/BuildingsAll.ts"
buildings_ts_string = "export const jsonBuildingsAll"
buildings_smaller_ts_output = "../../src/components/scientific-viz/BuildingsAllSmaller.ts"
buildings_smaller_string = "export const jsonBuildingsAllSmaller"
buildings_clusters_ts_output = "../../src/components/scientific-viz/BuildingsClusters.ts"
buildings_clusters_ts_string = "export const jsonBuildingsClusters"
buildings_grouped_ts_output = "../../src/components/scientific-viz/BuildingsGroups.ts"
buildings_grouped_ts_string = "export const jsonBuildingsGroups"

# Call functions
generate_smaller_buildings(buildings_file_path, buildings_smaller_file_path)
process_geojson_buildings(buildings_file_path, receivers_file_path, all_buildings_json_path)
process_geojson_buildings(buildings_smaller_file_path, receivers_smaller_file_path, all_buildings_smaller_json_path)
# Restructure JSON for the iTwin
process_json_buildings(buildings_input, buildings_output, buildings_index_name, buildings_ranges)
process_json_buildings(buildings_smaller_input, buildings_smaller_output, buildings_smaller_index_name, buildings_smaller_ranges)
process_json_buildings(buildings_clusters_input, buildings_clusters_output, buildings_clusters_index_name, cluster_ranges)
process_json_buildings(buildings_grouped_input, buildings_grouped_output, buildings_grouped_index_name, buildings_grouped_ranges)
# Write .json data to .ts file
generate_ts_files(buildings_output, buildings_ts_output, buildings_ts_string)
generate_ts_files(buildings_smaller_output, buildings_smaller_ts_output, buildings_smaller_string)
generate_ts_files(buildings_clusters_output, buildings_clusters_ts_output, buildings_clusters_ts_string)
generate_ts_files(buildings_grouped_output, buildings_grouped_ts_output, buildings_grouped_ts_string)